In [6]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point, LineString
from busroute import snap_points_to_line

# busroute.shp 的 欄位名稱(因每次資料來源提供的會有所不同)
route_routename_col = 'RouteNameZ'
route_direction_col = 'Direction'

# Seq 的欄位名稱 (因每次資料來源提供的會有所不同)
seq_routename_col = 'RouteName'
seq_direction_col = 'Direction'
seq_seq_col = 'Seq'
seq_lat_col = 'Lat'
seq_lng_col = 'Lon'

busroute = gpd.read_file(os.path.join(os.getcwd(),'..', 'input','Shp','BusRoute.shp'))
seq = seq = pd.read_csv(os.path.join(os.getcwd(),'..', 'input','seq.csv'))

In [7]:
# 只做同時有站序 & 路線檔案的
routelist = list(set(list(busroute[route_routename_col])) & set(list(seq[seq_routename_col])))
print("可計算的路線共有:", len(routelist),'條')
only_in_route = list(set(busroute[route_routename_col]) - set(seq[seq_routename_col]))
print("只有路線檔案的路線:", only_in_route)
only_in_seq = list(set(seq[seq_routename_col]) - set(busroute[route_routename_col]))
print("只有站序檔案的路線:", only_in_seq)

可計算的路線共有: 379 條
只有路線檔案的路線: []
只有站序檔案的路線: []


In [8]:
shp_outputfolder = os.path.join(os.getcwd(),'..', 'output','shp')
os.makedirs(shp_outputfolder, exist_ok=True)

以下嘗試

In [86]:
route = routelist[4]
print("現在示範的路線編號為：", route)

directions = [0,1]
direction = directions[0]
print("現在示範的方向為：", direction)

現在示範的路線編號為： 5038
現在示範的方向為： 0


In [106]:
busroute_select = busroute[ (busroute[route_routename_col] == route) & (busroute[route_direction_col] == direction)][[route_routename_col,route_direction_col,'geometry' ]].reset_index(drop = True)
seq_select = seq[ (seq[seq_routename_col] == route) & (seq[seq_direction_col] == direction) ].sort_values(seq_seq_col).reset_index(drop = True)
# 假設 seq_select 包含 Lat 和 Lon 欄位，創建一個 geometry 欄位
seq_select['geometry'] = seq_select.apply(lambda row: Point(row[seq_lng_col], row[seq_lat_col]), axis=1)
# 將 Pandas DataFrame 轉換為 GeoDataFrame
seq_select = gpd.GeoDataFrame(seq_select, geometry='geometry').drop_duplicates(subset=[seq_seq_col]).reset_index(drop = True)
seq_select = seq_select.set_crs(epsg=4326, inplace=True)

In [107]:
# busroute_select.to_file(os.path.join(shp_outputfolder, 'busroute_select.shp'))
# seq_select.to_file(os.path.join(shp_outputfolder, 'seq_select.shp'))

In [108]:
# (1) 把公車站點（投影到路線上）
# 假設 busroute_select 和 seq_select 已經是 GeoDataFrames，並且你已經知道欄位名稱
seq_select = snap_points_to_line(seq_select, busroute_select, 
                                         route_routename_col=route_routename_col, 
                                         route_direction_col=route_direction_col, 
                                         seq_routename_col=seq_routename_col, 
                                         seq_direction_col=seq_direction_col, 
                                         seq_lat_col=seq_lat_col, 
                                         seq_lng_col=seq_lng_col)

(2) 把路線進行拆分

In [109]:
# busroute_select = busroute_select.rename(columns = {route_routename_col:'RouteName',route_direction_col:'Direction'})
# seq_select = seq_select.rename(columns = {seq_routename_col:'RouteName', seq_direction_col:'Direction', seq_seq_col:'Seq'})

In [110]:
busroute_select

,RouteNameZ,Direction,geometry
0,5038,0.0,"LINESTRING (121.22389 24.95321, 121.2221 24.95..."


In [113]:
def split_routes(busroute_select, seq_select,
                 route_routename_col='RouteName',
                 route_direction_col='Direction',
                 seq_routename_col='RouteName',
                 seq_direction_col='Direction',
                 seq_seq_col='Seq',
                 seq_lat_col='Lat',
                 seq_lng_col='Lon'):
    output = []

    for _, route in busroute_select.iterrows():
        route_name = route[route_routename_col]
        direction = route[route_direction_col]
        geometry = route['geometry']

        # 過濾對應路線與方向的站點
        stops = seq_select[(seq_select[seq_routename_col] == route_name) & 
                           (seq_select[seq_direction_col] == direction)].sort_values(seq_seq_col)

        # 確保站點順序對應於路線
        stop_coords = [(row[seq_lng_col], row[seq_lat_col]) for _, row in stops.iterrows()]

        for i in range(len(stop_coords) - 1):
            start_point = Point(stop_coords[i])
            end_point = Point(stop_coords[i + 1])

            # 找到站點在路線中的比例位置
            start_distance = geometry.project(start_point)
            end_distance = geometry.project(end_point)

            # 提取路線幾何分段
            segment = substring(geometry, start_distance, end_distance)

            output.append({
                'RouteName': route_name,
                'Direction': direction,
                'StartSeq': stops.iloc[i][seq_seq_col],
                'EndSeq': stops.iloc[i + 1][seq_seq_col],
                'geometry': segment
            })

    return gpd.GeoDataFrame(output)

# 執行範例
result = split_routes(busroute_select, seq_select, route_routename_col = 'RouteNameZ')
print(result)


   RouteName  Direction  StartSeq  EndSeq  \
0       5038        0.0         1       2   
1       5038        0.0         2       3   
2       5038        0.0         3       4   
3       5038        0.0         4       5   
4       5038        0.0         5       6   
5       5038        0.0         6       7   
6       5038        0.0         7       8   
7       5038        0.0         8       9   
8       5038        0.0         9      10   
9       5038        0.0        10      11   
10      5038        0.0        11      12   
11      5038        0.0        12      13   
12      5038        0.0        13      14   
13      5038        0.0        14      15   
14      5038        0.0        15      16   
15      5038        0.0        16      17   
16      5038        0.0        17      18   
17      5038        0.0        18      19   
18      5038        0.0        19      20   
19      5038        0.0        20      21   
20      5038        0.0        21      22   
21      50

In [111]:
def split_routes(busroute_select, seq_select,
                 route_routename_col='RouteName',
                 route_direction_col='Direction',
                 seq_routename_col='RouteName',
                 seq_direction_col='Direction',
                 seq_seq_col='Seq',
                 seq_lat_col='Lat',
                 seq_lng_col='Lon'):
    output = []

    for _, route in busroute_select.iterrows():
        route_name = route[route_routename_col]
        direction = route[route_direction_col]
        geometry = route['geometry']

        # 過濾對應路線與方向的站點
        stops = seq_select[(seq_select[seq_routename_col] == route_name) & 
                           (seq_select[seq_direction_col] == direction)].sort_values(seq_seq_col)

        # 確保站點順序對應於路線
        stop_coords = [(row[seq_lng_col], row[seq_lat_col]) for _, row in stops.iterrows()]

        for i in range(len(stop_coords) - 1):
            start_point = Point(stop_coords[i])
            end_point = Point(stop_coords[i + 1])

            # 找到站點在路線中的比例位置
            start_distance = geometry.project(start_point)
            end_distance = geometry.project(end_point)

            # 提取路線幾何分段
            segment = substring(geometry, start_distance, end_distance)

            output.append({
                'RouteName': route_name,
                'Direction': direction,
                'StartSeq': stops.iloc[i][seq_seq_col],
                'EndSeq': stops.iloc[i + 1][seq_seq_col],
                'geometry': segment
            })
            output  = gpd.GeoDataFrame(output)
            # output['OD'] = output['StartSeq'].astype('str') + "-" + output['EndSeq']
            # output = output.reindex(columns = ['RouteName','OD' , 'Direction', 'StartSeq', 'EndSeq', 'geometry'])
    return gpd.GeoDataFrame(output)
    # return output

# 執行範例
result = split_routes(busroute_select, seq_select, route_routename_col = 'RouteNameZ')
print(result)


AttributeError: 'GeoDataFrame' object has no attribute 'append'

In [105]:
# # busroute.shp 的 欄位名稱(因每次資料來源提供的會有所不同)
# route_routename_col = 'RouteNameZ'
# route_direction_col = 'Direction'

# # Seq 的欄位名稱 (因每次資料來源提供的會有所不同)
# seq_routename_col = 'RouteName'
# seq_direction_col = 'Direction'
# seq_seq_col = 'Seq'
# seq_lat_col = 'Lat'
# seq_lng_col = 'Lon'

# def split_routes(busroute_select, seq_select,route_routename_col = 'RouteName', route_direction_col = 'Direction', seq_routename_col = 'RouteName', seq_seq_col = 'Seq', seq_lat_col = 'Lat', seq_lng_col = 'Lon'):
#     import pandas as pd
#     import geopandas as gpd
#     from shapely.geometry import LineString, Point
#     from shapely.ops import substring

#     output = []

#     for _, route in busroute_select.iterrows():
#         route_name = route['RouteName']
#         direction = route['Direction']
#         geometry = route['geometry']

#         # 過濾對應路線與方向的站點
#         stops = seq_select[(seq_select['RouteName'] == route_name) & 
#                            (seq_select['Direction'] == direction)].sort_values('Seq')

#         # 確保站點順序對應於路線
#         stop_coords = [(row['Lon'], row['Lat']) for _, row in stops.iterrows()]

#         for i in range(len(stop_coords) - 1):
#             start_point = Point(stop_coords[i])
#             end_point = Point(stop_coords[i + 1])

#             # 找到站點在路線中的比例位置
#             start_distance = geometry.project(start_point)
#             end_distance = geometry.project(end_point)

#             # 提取路線幾何分段
#             segment = substring(geometry, start_distance, end_distance)

#             output.append({
#                 'RouteName': route_name,
#                 'StartSeq': stops.iloc[i]['Seq'],
#                 'EndSeq': stops.iloc[i + 1]['Seq'],
#                 'geometry': segment
#             })

#     return gpd.GeoDataFrame(output)



In [81]:
# def split_routes(busroute_select, seq_select):
#     import pandas as pd
#     import geopandas as gpd
#     from shapely.geometry import LineString, Point
#     from shapely.ops import substring

#     output = []

#     for _, route in busroute_select.iterrows():
#         route_name = route['RouteName']
#         direction = route['Direction']
#         geometry = route['geometry']

#         # 過濾對應路線與方向的站點
#         stops = seq_select[(seq_select['RouteName'] == route_name) & 
#                            (seq_select['Direction'] == direction)].sort_values('Seq')

#         # 確保站點順序對應於路線
#         stop_coords = [(row['Lon'], row['Lat']) for _, row in stops.iterrows()]

#         for i in range(len(stop_coords) - 1):
#             start_point = Point(stop_coords[i])
#             end_point = Point(stop_coords[i + 1])

#             # 找到站點在路線中的比例位置
#             start_distance = geometry.project(start_point)
#             end_distance = geometry.project(end_point)

#             # 提取路線幾何分段
#             segment = substring(geometry, start_distance, end_distance)

#             output.append({
#                 'RouteName': route_name,
#                 'StartSeq': stops.iloc[i]['Seq'],
#                 'EndSeq': stops.iloc[i + 1]['Seq'],
#                 'geometry': segment
#             })

#     return gpd.GeoDataFrame(output)



   RouteName  StartSeq  EndSeq  \
0       5038         1       2   
1       5038         2       3   
2       5038         3       4   
3       5038         4       5   
4       5038         5       6   
5       5038         6       7   
6       5038         7       8   
7       5038         8       9   
8       5038         9      10   
9       5038        10      11   
10      5038        11      12   
11      5038        12      13   
12      5038        13      14   
13      5038        14      15   
14      5038        15      16   
15      5038        16      17   
16      5038        17      18   
17      5038        18      19   
18      5038        19      20   
19      5038        20      21   
20      5038        21      22   
21      5038        22      23   
22      5038        23      24   
23      5038        24      25   
24      5038        25      26   
25      5038        26      27   
26      5038        27      28   
27      5038        28      29   
28      5038  

In [ ]:
# 執行切分
result = split_routes(busroute_select, seq_select)

In [83]:
result.crs

In [84]:
result = gpd.GeoDataFrame(result, geometry='geometry')
result = result.set_crs(epsg=4326, inplace=True)

In [85]:
result.to_file(os.path.join(shp_outputfolder, 'busroute_segment.shp'))